In [77]:
import requests as r
import re
from bs4 import BeautifulSoup as soup
import time
import pandas as pd
database=[]
preurl='sampleurl'

In [78]:
def send_message(header,time,place,state,story):
  chat_id = '871014841'
  api_key = '1098694743:AAH6S5LGU4aw1SI0_MLiyNyJ0Fuonx8AurE'
  msg = 'HEADLINE :\n{}\n\nTIME : {}\nPLACE : {}\nSECTION : {}\n\nSTORY :\n{}\n'.format(header,time,place,state,story)
  telegram_url = 'https://api.telegram.org/bot{}/sendMessage?text={}&chat_id={}'.format(api_key,msg,chat_id)
  telegram_response = r.get(telegram_url)
  print('NEWS sent')

In [79]:
def scrape(url,sname):
  StateName=sname
  LINK=[]
  LINK.append(url)
  regex = re.compile('.*content-body-.*')
  CC=[regex]
  fdata=[]
  data=r.get(LINK[0])
  page_data=soup(data.text,'html.parser')
  Sectional=page_data.find('a',{'class':'section-name'}).text
  Header=page_data.find('h1',{'class':'title'}).text
  dp=page_data.find('div',{'class':'ut-container'})
  pad=dp.find_all('span',{'class':'blue-color ksl-time-stamp'})
  if(len(pad)==2): 
    Place=pad[0].text
    Place = Place.replace(',', '')
    Date=pad[1].text
  elif(len(pad)==1):
    Place='India'
    Date=pad[0].text
  Content=page_data.find('div',{'id':CC[0]}).text
  column=[StateName,Date,Sectional,Place,Header,Content]
  news=[]
  for line in column:
    news.append(line.strip().split('\n'))
  for k in range(0,len(news)):
    temp=news[k][0]
    fdata.append(temp)
  send_message(fdata[4],fdata[1],fdata[3],fdata[2],fdata[5])
  database.append(fdata)

In [80]:
def state():
  statenamelist=['andhra-pradesh','karnataka','kerala','tamil-nadu','telangana','other-states']
  for statename in statenamelist:
    url='https://www.thehindu.com/news/national/'+ statename +'/'
    #print(url)
    data=r.get(url)
    page_data=soup(data.text,'html.parser')
    jis=page_data.find('div',{'class':'justIn-story'})
    jis1=jis.find('a')
    jisurl=jis1.get('href')
    jist=jis1.text
    jist=jist.replace('\n', ' ')
    jist=jist.upper()
    jist=jist.strip().split(' ')
    keywords=['COVID-19','CORONAVIRUS','QUARANTINE','LOCKDOWN','CORONA','']
    flag=0
    for i in jist:
      for j in keywords:
        if(i==j and flag==0):
          flag=1
          try:
            scrape(jisurl,statename)
          except:
            print('ERROR\n')
    if (flag==0):
      print('Not a COVID-19 News in ' + statename)
    else :
      print(data)

In [81]:
state()
df=pd.DataFrame(database)
#df

NEWS sent
<Response [200]>
NEWS sent
<Response [200]>
Not a COVID-19 News in kerala
NEWS sent
<Response [200]>
NEWS sent
<Response [200]>
NEWS sent
<Response [200]>
